# Labeled Stream Creator

## Environment

In [1]:
import nuclio

In [3]:
import os

base_path = os.path.abspath('../')
base_stream_path = f'/users/' + os.environ['V3IO_USERNAME']+ f'{base_path[5:]}'
data_path = os.path.join(base_path, 'data')
src_path = os.path.join(base_path, 'src')
streaming_path = os.path.join(base_stream_path, 'streaming')
fs_streaming_path = os.path.join(base_path, 'streaming')

os.environ['base_path'] = base_path
os.environ['data_path'] = data_path
os.environ['src_path'] = src_path
os.environ['streaming_path'] = streaming_path
os.environ['fs_streaming_path'] = os.path.join(base_path, 'streaming')

os.environ['METRICS_TABLE'] = fs_streaming_path + '/metrics'
os.environ['PREDICTIONS_TABLE'] = fs_streaming_path+'/predictions'
os.environ['OUTPUT_STREAM'] = streaming_path+'/labeled_stream'
os.environ['prediction_col'] = 'predictions'
os.environ['label_col'] = 'is_error'
os.environ['output_stream_shards'] = '1'
os.environ['BATCHES_TO_GENERATE'] = '20'

## Function

In [5]:
# nuclio: start-code

In [5]:
import os
import pandas as pd
import json
import v3io
import v3io.dataplane
import socket

In [6]:
def split_path(mntpath=''):
    if mntpath[0] == '/':
        mntpath = mntpath[1:]
    paths = mntpath.split('/')
    container = paths[0]
    subpath = ''
    if len(paths) > 1:
        subpath = mntpath[len(container):]
    return container, subpath

In [7]:
def create_stream(context, path, shards=1):
    # create a stream w shards
    container, stream_path = split_path(path)
    context.logger.info(f'Creating stream in Container: {container} & Path {stream_path}')
    response = context.v3io_client.stream.create(container=container,
                                        stream_path=stream_path, 
                                        shard_count=shards,
                                        raise_for_status=v3io.dataplane.RaiseForStatus.never)
    response.raise_for_status([409, 204])

In [8]:
def push_to_stream(context, stream_path, data):
    def restructure_stream_event(context, event):
        instances = [dict()]
        for key in data.keys():
            if key not in ['when', 'class', 'model', 'worker', 'hostname', context.prediction_col]:
                instances[0].update({key: event.pop(key)})
        event['request'] = {'instances': instances}
        event['resp'] = [int(event.pop(context.prediction_col))]
        return event
    
    records = json.loads(data.to_json(orient='records'))
    records = [{'data': json.dumps(restructure_stream_event(context, record))} for record in records]
    context.logger.info(f'Logging {len(records)} records, Record example: {records[0]}')
    container, stream_path = split_path(stream_path)
    # batch
    step = 10
    for idx in range(0, len(records), step):
        response = context.v3io_client.put_records(container=container,
                                                   path=stream_path, 
                                                   records=records[idx:idx+step])

In [9]:
def get_data_parquet(context, table, files_to_select=1):
    mpath = [os.path.join(table, file) for file in os.listdir(table) if file.endswith(('parquet', 'pq'))]
    files_by_updated = sorted(mpath, key=os.path.getmtime, reverse=False)
    context.logger.debug_with('Input', input_files=files_by_updated[:files_to_select])
    dfs = pd.concat([pd.read_parquet(file) for file in files_by_updated[:files_to_select]])
    return dfs

In [10]:
def init_context(context):
    
    # How many batches to create? (-1 will run forever)
    batches_to_generate = int(os.getenv('BATCHES_TO_GENERATE', 20))
    setattr(context, 'batches_to_generate', batches_to_generate)
    setattr(context, 'batches_generated', 0)
    
    # Set env vars
    setattr(context, 'metrics_table', os.environ['METRICS_TABLE'])
    setattr(context, 'predictions_table', os.environ['PREDICTIONS_TABLE'])
    setattr(context, 'output_stream', os.environ['OUTPUT_STREAM'])
    setattr(context, 'timestamp_col', os.getenv('timestamp_col', 'when'))
    setattr(context, 'orig_timestamp_col', os.getenv('orig_timestamp_col', 'timestamp'))
    
    v3io_client = v3io.dataplane.Client(logger_verbosity='DEBUG', transport_verbosity='DEBUG')
#     v3io_client.stream.create(container='users', stream_path='/orz/mlrun-demos/demos/network-operations/streaming/labeled_stream', shard_count=1)
    setattr(context, 'v3io_client', v3io_client)
    create_stream(context, context.output_stream)
    
    setattr(context, 'label_col', os.environ['label_col'])
    setattr(context, 'prediction_col', os.environ['prediction_col'])

In [11]:
def handler(context, event):
    
    # Limit the number of generated batches to save cluster resources
    # for people forgetting the demo running
    if (context.batches_to_generate == -1) or (context.batches_generated <= context.batches_to_generate):
        
        metrics = get_data_parquet(context, context.metrics_table, 2).loc[:, context.label_col].astype('int')
        metrics.index.names = list([name if name != context.orig_timestamp_col else context.timestamp_col for name in metrics.index.names])
        predictions = get_data_parquet(context, context.predictions_table, 2)
        context.logger.debug(f'Labeling metrics ({metrics.shape}) and predictions ({predictions.shape})')
        context.logger.debug_with('Indexes', metrics_index=metrics.index.names, predictions_index=predictions.index.names)

        print('metrics')
        print(metrics.head())
        print(type(metrics))
        metrics = pd.DataFrame(metrics)
        print('change')
        print(type(metrics))
        print(metrics.head())
        print(metrics.index.names)

        full_df = pd.merge(left=predictions, right=metrics, left_on=metrics.index.names, how='left', left_index=True, right_index=True)
        full_df = full_df.reset_index()
        context.logger.info(f'Fully labeled batch size is {full_df.shape}')
        context.logger.info(f'Indexes: {list(full_df.index.names)}')
        context.logger.info(f'Columns: {full_df.columns}')
        context.logger.info_with('sample', full_df=full_df.head(1))    
        push_to_stream(context, context.output_stream, full_df)
        
        # Update batches count
        context.batches_generated += 1

In [12]:
# nuclio: end-code

## Test

In [13]:
init_context(context)

Python> 2021-10-03 14:49:45,141 [info] Creating stream in Container: users & Path /dani/test/demos/network-operations/streaming/labeled_stream
2021-10-03 14:49:45,142 [debug] Tx: {'connection_idx': 0, 'method': 'POST', 'path': '/users/dani/test/demos/network-operations/streaming/labeled_stream/', 'headers': {'X-v3io-function': 'CreateStream', 'X-v3io-session-key': '5b4a006e-d1e7-4f0b-8101-3e35bba895e7', 'Content-Type': 'application/json'}, 'body': '{"ShardCount":1,"RetentionPeriodHours":24}'}
2021-10-03 14:49:45,143 [debug] Rx: {'connection_idx': 0, 'status_code': 204, 'body': b''}


In [14]:
event = nuclio.Event(body='')
out = handler(context, event)
out

FileNotFoundError: [Errno 2] No such file or directory: '/User/test/demos/network-operations/streaming/metrics'

## Stream test

In [25]:
from v3io.dataplane import Client
from pprint import pprint

In [26]:
v3io_client = Client()

In [27]:
# v3io_client.delete_stream(container='users', path='/admin/demos/network-operations/streaming/labeled_stream')

In [28]:
def print_stream(path, shard='0', seek_type='EARLIEST', last=100):
    # seek the shard to the first record in it
    container, stream_path = split_path(path)
    shard_path = os.path.join(stream_path, shard)
    response = v3io_client.seek_shard(container=container,
                                      path=shard_path, 
                                      seek_type=seek_type)
    response.raise_for_status()

    # get records, starting from the location we got from seek
    response = v3io_client.get_records(container=container,
                                       path=shard_path, 
                                       location=response.output.location)
    response.raise_for_status()
    
    models = ['pagehinkley', 'eddm', 'ddm']
    result_record = response.output.records
    records = [json.loads(record.data) for record in result_record[:last]]
    pprint(records)

In [30]:
print_stream(context.output_stream, seek_type='EARLIEST', last=2)

[{'class': 'RandomForestClassifier',
  'hostname': 'jupyter-558bf7fbc8-sq5kd',
  'model': 'netops_predictor_v1',
  'request': {'instances': [{'company': 'Wilson_LLC',
                             'cpu_utilization': 66.9391393542,
                             'data_center': 'Zachary_Drives',
                             'device': '6001003522699',
                             'is_error': 0,
                             'latency': 0.5372793066,
                             'packet_loss': 0.0,
                             'throughput': 256.4821896882}]},
  'resp': [0],
  'when': 1593499337454,
  'worker': None},
 {'class': 'RandomForestClassifier',
  'hostname': 'jupyter-558bf7fbc8-sq5kd',
  'model': 'netops_predictor_v1',
  'request': {'instances': [{'company': 'Wilson_LLC',
                             'cpu_utilization': 72.4927066691,
                             'data_center': 'Obrien_Mountain',
                             'device': '0966571261270',
                             'is_er

## Deploy

In [84]:
from mlrun import code_to_function, mount_v3io

In [85]:
fn = code_to_function('labeled-stream-creator',
                      kind='nuclio',
                      project='network-operations', image='mlrun/ml-models')
fn.spec.build.commands = ['pip install v3io']
fn.apply(mount_v3io())
fn.add_trigger('cron', nuclio.triggers.CronTrigger(interval='1m'))
fn.set_envs({'METRICS_TABLE' : fs_streaming_path + '/metrics'
             'PREDICTIONS_TABLE' : fs_streaming_path+'/predictions'
             'OUTPUT_STREAM' : streaming_path+'/labeled_stream'
             'prediction_col' : 'predictions'
             'label_col' : 'is_error'
             'output_stream_shards' : '1'
             'BATCHES_TO_GENERATE' : '20'})

In [86]:
fn.save()
fn.export('../src/labeled_stream_creator.yaml')

> 2020-12-22 12:53:18,092 [info] function spec saved to path: ../src/labeled_stream_creator.yaml


In [87]:
fn.deploy(project='network-operations')

> 2020-12-22 12:53:18,098 [info] Starting remote function deploy
2020-12-22 12:53:18  (info) Deploying function
2020-12-22 12:53:18  (info) Building
2020-12-22 12:53:18  (info) Staging files and preparing base images
2020-12-22 12:53:18  (info) Building processor image
2020-12-22 12:53:22  (info) Build complete
2020-12-22 12:53:28  (info) Function deploy complete
> 2020-12-22 12:53:28,970 [info] function deployed, address=default-tenant.app.lewpwntlsyrb.iguazio-cd1.com:32619


'http://default-tenant.app.lewpwntlsyrb.iguazio-cd1.com:32619'

In [29]:
predictions = pd.read_parquet('/User/demos/network-operations/streaming/predictions/20200630T064217-20200630T074212.parquet')
predictions

cpu_utilization  \
when                    company                     data_center      device        model               class                  worker hostname                                    
2020-06-30 06:42:17.454 Wilson_LLC                  Zachary_Drives   6001003522699 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        66.939139   
                                                    Obrien_Mountain  0966571261270 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        72.492707   
                                                                     8069812479542 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        69.116878   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        64.944107   
                                                                     4285071567351 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        78.641128   
...                                                                                                                                                                        ...   
2020-06-30 07:42:12.454 Wilson_LLC                  Obrien_Mountain  8069812479542 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        59.574487   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd       100.000000   
                                                                     4285071567351 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd       100.000000   
                                                    Dominique_Branch 4579248894449 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        69.053014   
                                                                     7731097392608 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd        69.629988   

                                                                                                                                                                  latency  \
when                    company                     data_center      device        model               class                  worker hostname                               
2020-06-30 06:42:17.454 Wilson_LLC                  Zachary_Drives   6001003522699 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    0.537279   
                                                    Obrien_Mountain  0966571261270 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    0.000000   
                                                                     8069812479542 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    2.606934   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    1.571046   
                                                                     4285071567351 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    0.000000   
...                                                                                                                                                                   ...   
2020-06-30 07:42:12.454 Wilson_LLC                  Obrien_Mountain  8069812479542 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd    0.000000   
                        Bennett__Delacruz_and_Walls Natasha_Harbors  5863502247054 netops_predictor_v1 RandomForestClassifier NaN    jupyter-558bf7fbc8-sq5kd  100.000000   
                                                                     4285071567351 netops_predictor_v1 RandomFores